# FashionMNIST Challenge 之 KNN算法實現

## 一、算法原理

# PASS

## 二、編程實現
### 1、设计并训练KNN算法对图片进行分类。

In [1]:
import numpy as np
import tensorflow as tf


def read_tfrecord(filename, tensor=[1, 784], num=5000):
    """
    讀取tfrecord文件數據
    :param filename: 文件名
    :param tensor: 緯度
    :param num: 讀取數據數量
    :return: images 圖片數據列表    labels 標籤列表
    """
    filename_queue = tf.train.string_input_producer([filename])
    reader = tf.TFRecordReader()
    _, serialized_example = reader.read(filename_queue)
    features = tf.parse_single_example(serialized_example,
                                       features={
                                           'label': tf.FixedLenFeature([], tf.int64),
                                           'img_val': tf.FixedLenFeature(tensor, tf.float32),
                                       })
    image = tf.cast(features['img_val'], tf.float64)
    label = tf.cast(features['label'], tf.int32)
    images = []
    labels = []
    with tf.Session() as sess:
        init_op = tf.global_variables_initializer()
        sess.run(init_op)
        coord = tf.train.Coordinator()
        threads = tf.train.start_queue_runners(coord=coord)
        for i in range(num):
            example, l = sess.run([image, label])
            images.append(np.array(example[0]))
            tem = np.zeros((1, 10))
            tem[0][l] = 1.0
            labels.append(tem[0])
            del tem
        coord.request_stop()
        coord.join(threads=threads)
    images = np.array(images)
    labels = np.array(labels)
    return images, labels


In [2]:
def KNN():
    """
    KNN核心算法
    :return: 計算準確率
    """
    train_x, train_y = read_tfrecord(filename='train.tfrecords', tensor=[1, 784], num=60000)
    test_x, test_y = read_tfrecord(filename='test.tfrecords', tensor=[1, 784], num=5000)
    # 數據二值化處理
    #     for i in range(len(train_x)):
    #         for j in range(len(train_x[0])):
    #             if train_x[i][j]>0:
    #                 train_x[i][j]=1.0
    #     print(train_x[0])
    #     for i in range(len(test_x)):
    #         for j in range(len(test_x[0])):
    #             if test_x[i][j] > 0:
    #                 test_x[i][j] = 1.0
    xtr = tf.placeholder(tf.float32, [None, 784])
    xte = tf.placeholder(tf.float32, [784])
    distance = tf.sqrt(tf.reduce_sum(tf.pow(tf.add(xtr, tf.negative(xte)), 2), reduction_indices=1))
    pred = tf.argmin(distance, 0)
    init = tf.global_variables_initializer()
    sess = tf.Session()
    sess.run(init)


    right = 0
    for i in range(250):
        if i % 50 == 0 and i != 0:
            print("已处理 {0}，正确率为{1}".format(i, right / i))
        ansIndex = sess.run(pred, {xtr: train_x, xte: test_x[i, :]})
        # print('prediction is ', str(np.where(train_y[ansIndex]==np.max(train_y[ansIndex]))))
        # print('true value is ', str(np.where(test_y[i]==np.max(test_y[i]))))
        if np.argmax(test_y[i]) == np.argmax(train_y[ansIndex]):
            right += 1.0
    accracy = right / 250.0
    print(accracy)


In [3]:
if __name__ == "__main__":
    KNN()

已处理 50，正确率为0.78
已处理 100，正确率为0.84
已处理 150，正确率为0.84
已处理 200，正确率为0.855
0.86


## 三、算法總結
1. 嗯，等會再寫總結，懶～～

#### 完成時間：2017年11月15日，完成人：土豆豆